In [1]:
%matplotlib inline
import xarray as xr
import os
import numpy as np
import matplotlib.pyplot as plt
import collections
import warnings 
from netCDF4 import default_fillvals
from scipy.stats import hmean

# import soil functions
from parameter_functions import (is_soil_class, is_param_value, classify_soil_texture)

# import veg functions
from parameter_functions import (calculate_cv_pft, calculate_nveg_pfts, map_pft_to_nldas_class, is_overstory, 
                                 calc_root_fract, calc_root_depth_rz1, calc_root_depth_rz2)
# import soil layer aggregation functions 
from parameter_functions import (calculate_first_layer_harmonic_mean, calculate_second_layer_harmonic_mean, 
                                 calculate_third_layer_harmonic_mean, calculate_first_layer_arithmetic_mean, 
                                 calculate_second_layer_arithmetic_mean, calculate_third_layer_arithmetic_mean)

from parameter_functions_v2 import (soil_class_values, calculate_init_moist, 
                                    calculate_baseflow_parameters)

# define fillvals
fillval_f = default_fillvals['f8']
fillval_i = default_fillvals['i4']

This is a notebook for deriving new VIC 5 parameters in the RASM domain at multiple resolutions. 

Currently 50km (`wr50a_ar9v`) and 25km (`wr50a_ar9v`) are supported. 

The accompanying excel sheet `deriving_new_parameters_v2.xlsx` and Word Doc `procedure_for_derivation.docx` provide additional citations and details on methods. 

__Set resolution for filenames__

In [9]:
res = '25km'

# set file extensions 
if res == "50km":
    grid = 'wr50a_ar9v4'
elif res == "25km":
    grid = 'wr25b_ar9v4'
elif res == "10km":
    raise ValueError("we do not have a domain file at %s so this option has not been implemented" %res)
print("calculating parameters at %s" %res)

calculating parameters at 25km


__Set domain file__

In [5]:
if res == '50km':
    domain = xr.open_dataset(os.path.join('/u/home/gergel/data/parameters', 
                                      'domain.lnd.wr50a_ar9v4.100920.nc'))
elif res == "25km":
    domain = xr.open_dataset(os.path.join('/u/home/gergel/data/parameters', 
                                      'domain.lnd.wr25b_ar9v4.170413.nc'))

In [28]:
old_params = xr.open_dataset(os.path.join('/u/home/gergel/data', 
                                          'vic_params_wr50a_vic5.0.dev_20160328.nc'))

__Options__

In [2]:
# if set to True, add additional organic_fract options to parameter file, including soil organic fraction, 
# soil particle density of OM, and bulk density of OM 
organic_fract = False
bulk_density_comb = True

__Load soil data__

In [7]:
soil_data_vars = collections.OrderedDict()
soil_data_vars['silt'] = 'silt_sl*'
soil_data_vars['sand'] = 'sand_sl*'
soil_data_vars['clay'] = 'clay_sl*'
soil_data_vars['bulk_density'] = 'bulk_density_sl*'
soil_data_vars['organic_fract'] = 'organic_fract_sl*'

# soil data dict with nlayer = 7 (base resolution of data)
soil_data = {}
soil_data_dir = '/u/home/gergel/data/parameters/soil_data/rasm_grid_netcdfs/%s' %res
for soil_var, soil_wildcard in soil_data_vars.items(): 
    soil_data[soil_var] = xr.open_mfdataset(os.path.join(soil_data_dir, soil_wildcard),
                                            concat_dim='nlayer', 
                                            data_vars='all', 
                                            coords='all')

load CLM PFTs to use for vegetation parameters

In [10]:
# load regridded PFTs 
pfts_data_dir = '/u/home/gergel/data/parameters/pfts/regridded_pfts'
pfts_filename = 'mksrf_landuse_rc2000_c110913_%s.nc' %grid
veg_data = xr.open_dataset(os.path.join(pfts_data_dir, pfts_filename))

calculate soil types based on percent clay, percent sand and bulk density


In [11]:
# classify_soil_texture(sand, clay, silt)
soil_type_array = xr.apply_ufunc(classify_soil_texture, 
                                 soil_data['sand']['sand'].where(domain.mask == 1), 
                                 soil_data['clay']['clay'].where(domain.mask == 1),  
                                 soil_data['silt']['silt'].where(domain.mask == 1),
                                 dask='allowed',
                                 vectorize=True)

In [12]:
ksat = xr.apply_ufunc(soil_class_values, 
                      soil_type_array,
                      'ksat',
                      dask='allowed',
                      vectorize=True)
quartz = xr.apply_ufunc(soil_class_values, 
                        soil_type_array,
                        'quartz',
                        dask='allowed',
                        vectorize=True)
Wcr_FRACT = xr.apply_ufunc(soil_class_values, 
                           soil_type_array,
                           'Wcr_FRACT',
                           dask='allowed',
                           vectorize=True)
Wpwp_FRACT = xr.apply_ufunc(soil_class_values, 
                            soil_type_array,
                           'Wpwp_FRACT',
                            dask='allowed',
                            vectorize=True)
b = xr.apply_ufunc(soil_class_values, 
                   soil_type_array,
                   'b',
                   dask='allowed',
                   vectorize=True)
bulk_density_min = xr.apply_ufunc(soil_class_values, 
                   soil_type_array,
                   'bulk_density',
                   dask='allowed',
                   vectorize=True)
resid_moist = xr.apply_ufunc(soil_class_values, 
                             soil_type_array,
                             'resid_moist',
                             dask='allowed',
                             vectorize=True)

load regridded GTOPO 30 data, data var for elevation is called `Band1`, in (m)

In [13]:
gtopo_filename = 'sdat_10003_1_20180525_151136146_%s.nc' %grid
gtopo = xr.open_dataset(os.path.join('/u/home/gergel/data/parameters/gtopo30', gtopo_filename))
elev = gtopo['Band1']

load regridded WORLDCLIM climate data for annual t and p 

In [14]:
clim_direc = '/u/home/gergel/data/parameters/world_clim_data/%s' %res
prec = xr.open_mfdataset(os.path.join(clim_direc, 'prec*'),
                                      concat_dim='time', 
                                      data_vars=['prec'], 
                                      coords='all')

# aggregate to annual, need average annual precip
annual_precip = prec['prec'].sum('time')

temp = xr.open_mfdataset(os.path.join(clim_direc, 'tavg*'),
                                      concat_dim='time', 
                                      data_vars='all', 
                                      coords='all')
tavg = temp['tavg'].mean('time')

calculate Cv from PFTs

In [15]:
cv = xr.apply_ufunc(calculate_cv_pft, 
                    veg_data['PCT_PFT'].where(domain.mask == 1),
                    dask='allowed',
                    vectorize=True)

calculate number of active PFTs, `Nveg` 

In [16]:
nvegg = xr.apply_ufunc(calculate_nveg_pfts,
                      veg_data['PCT_PFT'].where(domain.mask == 1),
                      dask='allowed',
                      input_core_dims=[['pft']],
                      vectorize=True)
Nveg = nvegg - 1

import LAI and vegetation height, `MONTHLY_LAI` and `MONTHLY_HEIGHT_TOP`

In [17]:
lai_file = xr.open_dataset(os.path.join('/u/home/gergel/data/parameters/lai/regridded_lai', 
                                   'mksrf_lai_78pfts_simyr2005.c170413_%s_lai.nc' %grid))

In [18]:
veg_height_file = xr.open_dataset(os.path.join('/u/home/gergel/data/parameters/lai/regridded_lai', 
                                'mksrf_lai_78pfts_simyr2005.c170413_%s_veg_height.nc' %grid))

LAI and veg_height from CLM and `PCT_PFT` from CLM have a different number of PFTs (`PCT_PFT` has one more PFT, 17 vs 16). The extra PFT in `PCT_PFT` has `PCT_PFT` = 0 over the entire RASM domain, so I just slice the LAI and veg_height from the 0th PFT (water/bare soil) and concatenate it for the 16th PFT. 

In [19]:
lai_slice = lai_file['MONTHLY_LAI'].isel(pft = 0)
vegheight_slice = veg_height_file['MONTHLY_HEIGHT_TOP'].isel(pft=0)

In [20]:
lai = xr.concat([lai_file['MONTHLY_LAI'], lai_slice], dim='pft')
veg_height = xr.concat([veg_height_file['MONTHLY_HEIGHT_TOP'], vegheight_slice], dim='pft')

`veg_rough` = 0.123 * `veg_height`

`displacement` = 0.67 * `veg_height`

In [21]:
veg_rough = 0.123 * veg_height
displacement = 0.67 * veg_height

displacement.values[displacement.values == 0] = 1.0

map albedo, root zone fraction and root zone depth based on vegetation type. see `deriving_new_parameters_v2.xlsx` sheet titled `PFT-NLDAS Mapping` for mapping between NLDAS vegetation classes (used in old VIC 5 parameters) and CLM PFTs. This mapping is based on obvious relationships and some approximations (used for PFTs 8-11).

get albedo values from VIC 5 parameters for NLDAS classes, map to CLM PFTs

create array that is the size of (nj, ni, pft, root_zone) to operate on

In [25]:
masknan_vals = domain['mask'].where(domain['mask'] == 1).values
if res == "50km":
    nj = 205
    ni = 275
elif res == "25km":
    nj = 413
    ni = 551
    
num_veg = 17


arr_months = np.rollaxis(np.dstack((masknan_vals, masknan_vals, masknan_vals, masknan_vals, 
                                    masknan_vals, masknan_vals, masknan_vals, masknan_vals, 
                                    masknan_vals, masknan_vals, masknan_vals, masknan_vals)), 
                        axis=2)
arr_nlayer = np.rollaxis(np.dstack((masknan_vals, masknan_vals, masknan_vals)), 
                        axis=2)

arr_rootzone = np.rollaxis(np.dstack((masknan_vals, masknan_vals)), 
                        axis=2)

arr_veg_classes = np.rollaxis(np.dstack((masknan_vals, masknan_vals, masknan_vals, masknan_vals, 
                                         masknan_vals, masknan_vals, masknan_vals, masknan_vals, 
                                         masknan_vals, masknan_vals, masknan_vals, masknan_vals, 
                                         masknan_vals, masknan_vals, masknan_vals, masknan_vals, 
                                         masknan_vals)), 
                              axis=2)
arr_veg_classes_rootzone = np.vstack((arr_rootzone, arr_rootzone, arr_rootzone, arr_rootzone, 
                                      arr_rootzone, arr_rootzone, arr_rootzone, arr_rootzone, 
                                      arr_rootzone, arr_rootzone, arr_rootzone, arr_rootzone, 
                                      arr_rootzone, arr_rootzone, arr_rootzone,
                                      arr_rootzone, arr_rootzone)).reshape(num_veg, 2, nj, ni)
arr_veg_classes_month = np.vstack((arr_months, arr_months, arr_months, arr_months, arr_months, 
                                   arr_months, arr_months, arr_months, arr_months, arr_months, 
                                   arr_months, arr_months, arr_months, arr_months, arr_months, 
                                   arr_months, arr_months,)).reshape(num_veg, 12, nj, ni)

create Dataset for variables and define data_vars 

change dims and order of dims of LAI array 

In [26]:
lai = lai.rename({'time': 'month', 'pft': 'veg_class'})
lai = lai.transpose('veg_class', 'month', 'nj', 'ni')

veg_rough = veg_rough.rename({'time': 'month', 'pft': 'veg_class'})
veg_rough = veg_rough.transpose('veg_class', 'month', 'nj', 'ni')

displacement = displacement.rename({'time': 'month', 'pft': 'veg_class'})
displacement = displacement.transpose('veg_class', 'month', 'nj', 'ni')

In [29]:
params = xr.Dataset()

# assign veg class indexing
params['veg_class'] = xr.DataArray(np.arange(1, 18), dims='veg_class', 
                                   attrs={'long_name': "vegetation class"})
params['nlayer'] = xr.DataArray(np.arange(0, 3), dims='nlayer')

params['Cv'] = xr.DataArray(cv,
                                 dims=('veg_class','nj', 'ni'),
                                 coords={'xc': domain.xc, 'yc': domain.yc},
                                 attrs={'description': "Fraction of grid cell covered by vegetation tile",
                                        'units': "fraction", 'long_name': "Cv"},
                                 encoding={"_FillValue": fillval_f,
                                               "Coordinates": "xc yc"})

params['Nveg'] = xr.DataArray(Nveg,
                                   dims=('nj', 'ni'),
                                   coords={'xc': domain.xc, 'yc': domain.yc},
                                   attrs={'description': "Number of vegetation tiles in the grid cell", 
                                          'units': "N/A", 'long_name': "Nveg"},
                                   encoding={"_FillValue": fillval_i,
                                               "Coordinates": "xc yc", 'dtype': 'int32'})

params['trunk_ratio'] = xr.DataArray(arr_veg_classes,
                                 dims=('veg_class','nj', 'ni'),
                                 coords={'xc': domain.xc, 'yc': domain.yc},
                                 attrs={'description': "Ratio of total tree height that is trunk \
                                 (no branches) \
                                        The default value has been 0.2",
                                 'units': "fraction", 'long_name': "Cv"},
                                 encoding={"_FillValue": fillval_f,
                                               "Coordinates": "xc yc"})
params['rarc'] = xr.DataArray(arr_veg_classes,
                                 dims=('veg_class','nj', 'ni'),
                                 coords={'xc': domain.xc, 'yc': domain.yc},
                                 attrs={'description': "Architectural resistance of vegetation type \(~2 s/m)",
                                        'units': "s/m", 'long_name': "rarc"},
                                 encoding={"_FillValue": fillval_f,
                                               "Coordinates": "xc yc"})

params['rmin'] = xr.DataArray(np.copy(arr_veg_classes),
                                 dims=('veg_class','nj', 'ni'),
                                 coords={'xc': domain.xc, 'yc': domain.yc},
                                 attrs={'description': "Minimum stomatal resistance of vegetation type (~100 s/m)"},
                                 encoding={"_FillValue": fillval_f,
                                               "Coordinates": "xc yc"})
params['wind_h'] = xr.DataArray(np.copy(arr_veg_classes),
                                 dims=('veg_class','nj', 'ni'),
                                 coords={'xc': domain.xc, 'yc': domain.yc},
                                 attrs={'description': "Height at which wind speed is measured",
                                        'units': "m", 'long_name': "wind_h"},
                                 encoding={"_FillValue": fillval_f,
                                               "Coordinates": "xc yc"})

params['RGL'] = xr.DataArray(np.copy(arr_veg_classes),
                                 dims=('veg_class','nj', 'ni'),
                                 coords={'xc': domain.xc, 'yc': domain.yc},
                                 attrs={'description': "Minimum incoming shortwave radiation at which there will be \
                                        transpiration. For trees this is about 30 W/m^2, for crops about 100 W/m^2",
                                        'units': "W/m^2", 'long_name': "RGL"},
                                 encoding={"_FillValue": fillval_f,
                                               "Coordinates": "xc yc"})

params['rad_atten'] = xr.DataArray(arr_veg_classes,
                                 dims=('veg_class','nj', 'ni'),
                                 coords={'xc': domain.xc, 'yc': domain.yc},
                                 attrs={'description': "Radiation attenuation factor. Normally set to 0.5, though may \
                                        need to be adjusted for high latitudes",
                                        'units': "fraction", 'long_name': "rad_atten"},
                                 encoding={"_FillValue": fillval_f,
                                               "Coordinates": "xc yc"})

params['wind_atten'] = xr.DataArray(arr_veg_classes,
                                 dims=('veg_class','nj', 'ni'),
                                 coords={'xc': domain.xc, 'yc': domain.yc},
                                 attrs={'description': "Wind speed attenuation through the overstory. The default value \
                                        has been 0.5",
                                        'units': "fraction", 'long_name': "wind_atten"},
                                 encoding={"_FillValue": fillval_f,
                                               "Coordinates": "xc yc"})
params['max_snow_albedo'] = xr.DataArray(np.copy(arr_veg_classes),
                                 dims=('veg_class','nj', 'ni'),
                                 coords={'xc': domain.xc, 'yc': domain.yc},
                                 attrs={'description': "maximum snow albedo from Barlage et al 2005",
                                        'units': "fraction", 'long_name': "max_snow_albedo"},
                                 encoding={"_FillValue": fillval_f,
                                               "Coordinates": "xc yc"})
params['albedo'] = xr.DataArray(arr_veg_classes_month,
                                         dims=('veg_class','month','nj', 'ni'),
                                         coords={'xc': domain.xc, 'yc': domain.yc},
                                         attrs={'description': "Shortwave albedo for vegetation type",
                                                'units': "fraction", 'long_name': "albedo"},
                                         encoding={"_FillValue": fillval_f,
                                               "Coordinates": "xc yc"})

params['LAI'] = xr.DataArray(lai.values.reshape(num_veg, 12, nj, ni),
                                 dims=('veg_class','month','nj', 'ni'),
                                 coords={'xc': domain.xc, 'yc': domain.yc, 'month': old_params.month},
                                 attrs={'description': "Leaf Area Index, one per month",
                                        'units': "N/A", 'long_name': "LAI"},
                                 encoding={"_FillValue": fillval_f,
                                               "Coordinates": "xc yc"})
params['overstory'] = xr.DataArray(arr_veg_classes,
                                 dims=('veg_class','nj', 'ni'),
                                 coords={'xc': domain.xc, 'yc': domain.yc},
                                 attrs={'description': "Flag to indicate whether or not the current vegetation type \
                                        has an overstory (TRUE for overstory present (e.g. trees), FALSE for \
                                        overstory not present (e.g. grass))",
                                        'units': "N/A", 'long_name': "overstory"},
                                 encoding={"_FillValue": fillval_i,
                                               "Coordinates": "xc yc", 'dtype': 'int32'})
# time: 12, pft: 17, nj: 205, ni: 275
params['displacement'] = xr.DataArray(displacement.values.reshape(num_veg, 12, nj, ni), 
                                         dims=('veg_class','month','nj', 'ni'),
                                         coords={'month': old_params['month'], 'xc': domain.xc, 
                                                 'yc': domain.yc},
                                         attrs={'description': "Vegetation displacement height (typically 0.67 \
                                                * vegetation height)",
                                                'units': "m", 'long_name': "displacement"},
                                         encoding={"_FillValue": fillval_f,
                                               "Coordinates": "xc yc"})
params['veg_rough'] = xr.DataArray(veg_rough.values.reshape(num_veg, 12, nj, ni),
                                         dims=('veg_class','month','nj', 'ni'),
                                         coords={'xc': domain.xc, 'yc': domain.yc},
                                         attrs={'description': "Vegetation roughness length (typically 0.123 \
                                                * vegetation height)",
                                                'units': "m", 'long_name': "veg_rough"},
                                         encoding={"_FillValue": fillval_f,
                                               "Coordinates": "xc yc"})

In [30]:
params['elev'] = xr.DataArray(elev.values,
                                dims=('nj', 'ni'),
                                coords={'xc': domain.xc, 'yc': domain.yc},
                                attrs={'description': "Average elevation of grid cell", 
                                              'units': "m", 'long_name': "elev"},
                                encoding={"_FillValue": fillval_f,
                                               "Coordinates": "xc yc"})
params['avg_T'] = xr.DataArray(tavg.values,
                                dims=('nj', 'ni'),
                                coords={'xc': domain.xc, 'yc': domain.yc},
                                attrs={'description': "Average soil temperature, used as the bottom boundary \
                                        for soil heat flux solutions", 
                                        'units': "C", 'long_name': "avg_T"},
                                encoding={"_FillValue": fillval_f,
                                               "Coordinates": "xc yc"})
params['annual_prec'] = xr.DataArray(annual_precip.values,
                                dims=('nj', 'ni'),
                                coords={'xc': domain.xc, 'yc': domain.yc},
                                attrs={'description': "Average annual precipitation", 
                                              'units': "mm", 'long_name': "annual_prec"},
                                encoding={"_FillValue": fillval_f,
                                               "Coordinates": "xc yc"})
roughness = np.copy(masknan_vals)
roughness[np.nonzero(masknan_vals)] = 0.001
params['rough'] = xr.DataArray(roughness,
                                dims=('nj', 'ni'),
                                coords={'xc': domain.xc, 'yc': domain.yc},
                                attrs={'description': "Surface roughness of bare soil", 
                                              'units': "m", 'long_name': "rough"},
                                encoding={"_FillValue": fillval_f,
                                               "Coordinates": "xc yc"})

define `trunk_ratio`, `rarc`, `rmin`, `wind_h`, `RGL`, `rad_atten`, `wind_atten`, `overstory`, `max_snow_albedo`

In [31]:
# trunk ratio, rarc, rad_atten
trunk_ratio = np.copy(arr_veg_classes)
params['trunk_ratio'].values = trunk_ratio * 0.2
# adjust for bare soil 
params['trunk_ratio'].values[0, :, :] = 0.0

rarc = np.copy(arr_veg_classes)
params['rarc'].values = rarc * 60
# adjust for bare soil
params['rarc'].values[0, :, :] = 100

rad_atten = np.copy(arr_veg_classes)
params['rad_atten'].values = rad_atten * 0.5
# adjust for bare soil 
params['rad_atten'].values[0, :, :] = 0.0

wind_atten = np.copy(arr_veg_classes)
params['wind_atten'].values = wind_atten * 0.5
# adjust for bare soil 
params['wind_atten'].values[0, :, :] = 0.0

In [32]:
# max_albedo
for pft in veg_data.pft.values:
    # get nldas mapping from pft
    nldas = map_pft_to_nldas_class(pft)
    if nldas == 0:
        max_alb = 0.34
    elif nldas == 1:
        max_alb = 0.37
    elif nldas == 2:
        max_alb = 0.35
    elif nldas == 3: 
        max_alb = 0.35
    elif nldas == 4: 
        max_alb = 0.44
    elif nldas == 5:
        max_alb = 0.69
    elif nldas == 6:
        max_alb = 0.43
    elif nldas == 7:
        max_alb = 0.56
    elif nldas == 8:
        max_alb = 0.70
    elif nldas == 9:
        max_alb = 0.65
    elif nldas == 10:
        max_alb = 0.46
    elif nldas == 11:
        max_alb = 0.84
    params['max_snow_albedo'].values[pft, :, :] = np.ones((1, nj, ni)) * max_alb

In [33]:
# rmin, wind_h
for pft in veg_data.pft.values:
    # get nldas mapping from pft
    nldas = map_pft_to_nldas_class(pft)
    if nldas >= 0 and nldas <= 3:
        rmin = np.asscalar(old_params.rmin.isel(veg_class=0).mean())
        wind_h = np.asscalar(old_params.wind_h.isel(veg_class=0).mean())
    elif nldas == 4:
        rmin = np.asscalar(old_params.rmin.isel(veg_class=4).mean())
        wind_h = np.asscalar(old_params.wind_h.isel(veg_class=4).mean())
    elif nldas >= 5 and nldas <= 6:
        rmin = np.asscalar(old_params.rmin.isel(veg_class=5).mean())
        wind_h = np.asscalar(old_params.wind_h.isel(veg_class=5).mean())
    elif nldas >= 7 and nldas <= 8:
        rmin = np.asscalar(old_params.rmin.isel(veg_class=7).mean())
        wind_h = np.asscalar(old_params.wind_h.isel(veg_class=7).mean())
    elif nldas == 9:
        rmin = np.asscalar(old_params.rmin.isel(veg_class=9).mean())
        wind_h = np.asscalar(old_params.wind_h.isel(veg_class=9).mean())
    elif nldas == 10:
        rmin = np.asscalar(old_params.rmin.isel(veg_class=10).mean())
        wind_h = np.asscalar(old_params.wind_h.isel(veg_class=10).mean())
    elif nldas == 11:
        rmin = np.asscalar(old_params.rmin.isel(veg_class=11).mean())
        wind_h = np.asscalar(old_params.wind_h.isel(veg_class=11).mean())
    params['rmin'].values[pft, :, :] = np.ones((1, nj, ni)) * rmin
    params['wind_h'].values[pft, :, :] = np.ones((1, nj, ni)) * wind_h

In [34]:
# RGL
for pft in veg_data.pft.values:
    # get nldas mapping from pft
    nldas = map_pft_to_nldas_class(pft)
    if nldas >= 0 and nldas <= 3:
        rgl = np.asscalar(old_params.wind_h.isel(veg_class=0).mean())
    elif nldas >= 4 and nldas <= 5:
        rgl = np.asscalar(old_params.wind_h.isel(veg_class=4).mean())
    elif nldas >= 6 and nldas <= 8:
        rgl = np.asscalar(old_params.wind_h.isel(veg_class=6).mean())
    elif nldas >= 9 and nldas <= 10:
        rgl = np.asscalar(old_params.wind_h.isel(veg_class=9).mean())
    elif nldas == 11:
        rgl = np.asscalar(old_params.wind_h.isel(veg_class=11).mean())
    params['RGL'].values[pft, :, :] = np.ones((1, nj, ni)) * rgl

In [35]:
# overstory
for pft in veg_data.pft.values:
    nldas = map_pft_to_nldas_class(pft)
    if nldas > 6:
        # no overstory
        overstory = 0
    else: 
        overstory = 1
    params['overstory'].values[pft, :, :] = np.ones((1, nj, ni)) * overstory

# adjust bare soil, it should obviously not have an overstory
params['overstory'].values[0, :, :] = np.ones((1, nj, ni)) * 0.0

In [36]:
root_depth_rz1 = xr.apply_ufunc(calc_root_depth_rz1,
                           params['Cv'].where(domain.mask == 1), 
                           dask='allowed',
                           vectorize=True)
root_depth_rz2 = xr.apply_ufunc(calc_root_depth_rz2,
                           params['Cv'].where(domain.mask == 1), 
                           dask='allowed',
                           vectorize=True)
root_depth = xr.concat([root_depth_rz1, root_depth_rz2],
                      dim='root_zone').transpose('veg_class', 'root_zone', 'nj', 'ni')

In [37]:
# root fract 

rz = 0
for pft in veg_data.pft.values:
    if pft == 0:
        root_fract_rz1 = xr.apply_ufunc(calc_root_fract,
                                        params['Cv'].isel(veg_class=pft),
                                        str(pft),
                                        str(rz),
                                        dask='allowed',
                                        vectorize=True)
    else: 
        root_fract_rz1 = xr.concat([root_fract_rz1, xr.apply_ufunc(calc_root_fract,
                                                                   params['Cv'].isel(veg_class=pft),
                                                                   str(pft),
                                                                   str(rz),
                                                                   dask='allowed',
                                                                   vectorize=True)],
                                  dim='veg_class')
rz = 1
for pft in veg_data.pft.values:
    if pft == 0:
        root_fract_rz2 = xr.apply_ufunc(calc_root_fract,
                                        params['Cv'].isel(veg_class=pft), 
                                        str(pft),
                                        str(rz),
                                        dask='allowed',
                                        vectorize=True)
    else: 
        root_fract_rz2 = xr.concat([root_fract_rz2, xr.apply_ufunc(calc_root_fract,
                                                                   params['Cv'].isel(veg_class=pft), 
                                                                   str(pft),
                                                                   str(rz),
                                                                   dask='allowed',
                                                                   vectorize=True)],
                                  dim='veg_class')
        
root_fract = xr.concat([root_fract_rz1, root_fract_rz2], dim='root_zone').transpose('veg_class', 'root_zone', 'nj', 'ni')

In [38]:
params['root_depth'] = xr.DataArray(root_depth,
                                         dims=('veg_class','root_zone','nj', 'ni'),
                                         coords={'xc': domain.xc, 'yc': domain.yc},
                                         attrs={'description': "Root zone thickness (sum of depths is total depth of \
                                                 root penetration)",
                                                'units': "m", 'long_name': "root_depth"},
                                         encoding={"_FillValue": fillval_f,
                                               "Coordinates": "xc yc"})
params['root_fract'] = xr.DataArray(root_fract,
                                         dims=('veg_class','root_zone','nj', 'ni'),
                                         coords={'xc': domain.xc, 'yc': domain.yc},
                                         attrs={'description': "Fraction of root in the current root zone",
                                                'units': "fraction", 'long_name': "root_fract"},
                                         encoding={"_FillValue": fillval_f,
                                               "Coordinates": "xc yc"})

__albedo__

In [39]:
# loop over pft classes and months 
for pft in veg_data.pft.values:
    for month in old_params.month.values: 
        nldas = map_pft_to_nldas_class(pft)
        albedo_slice = np.copy(arr_veg_classes_month[0, 0, :, :])
        albedo = np.asscalar(old_params.albedo.isel(veg_class=nldas).isel(month=month-1).mean())
        params['albedo'].values[pft, month - 1, :, :] = np.ones(shape=(1, 1, nj, ni)) * albedo

__load hydroclimate classes__

In [40]:
hydro_classes = xr.open_dataset(os.path.join('/u/home/gergel/data/parameters',
                                             'hydroclimate_masks_%s.nc' %grid))

__baseflow parameters: Ds, Dsmax, Ws__

In [ ]:
d1 = calculate_baseflow_parameters(domain, hydro_classes, "d1")
params['Ds'] = xr.DataArray(d1,
                              dims=('nj', 'ni'),
                              coords={'xc': domain.xc, 'yc': domain.yc},
                              attrs={'description': "Fraction of Dsmax where non-linear baseflow begins", 
                                         'units': "fraction", 'long_name': "Ds"},
                                 encoding={"_FillValue": fillval_f,
                                           "Coordinates": "xc yc"})
d2 = calculate_baseflow_parameters(domain, hydro_classes, "d2")
params['Dsmax'] = xr.DataArray(d2,
                              dims=('nj', 'ni'),
                              coords={'xc': domain.xc, 'yc': domain.yc},
                              attrs={'description': "Fraction of maximum soil moisture where non-linear baseflow occurs", 
                                              'units': "fraction", 'long_name': "Dsmax"},
                                 encoding={"_FillValue": fillval_f,
                                           "Coordinates": "xc yc"})
d3 = calculate_baseflow_parameters(domain, hydro_classes, "d3")
params['Ws'] = xr.DataArray(d3,
                              dims=('nj', 'ni'),
                              coords={'xc': domain.xc, 'yc': domain.yc},
                              attrs={'description': "Fraction of maximum soil moisture where non-linear baseflow occurs", 
                                              'units': "fraction", 'long_name': "Ws"},
                                 encoding={"_FillValue": fillval_f,
                                           "Coordinates": "xc yc"})
d4 = calculate_baseflow_parameters(domain, hydro_classes, "d4")
params['c'] = xr.DataArray(d4,
                           dims=('nj', 'ni'),
                           coords={'xc': domain.xc, 'yc': domain.yc},
                           attrs={'description': "Exponent used in baseflow curve, normally set to 2", 
                                      'units': "N/A", 'long_name': "c"},
                           encoding={"_FillValue": fillval_f,
                                  "Coordinates": "xc yc"})

b_i (`infilt`)

In [33]:
bi = np.copy(masknan_vals)
bi[np.nonzero(hydro_classes['arid'].values)] = 0.05
bi[np.nonzero(hydro_classes['temperate_dry'].values)] = 0.05
bi[np.nonzero(hydro_classes['cold_dry_perma'].values)] = 0.3
bi[np.nonzero(hydro_classes['cold_dry_noperma'].values)] = 0.5
bi[np.nonzero(hydro_classes['cold_wds_ws_perma'].values)] = 0.3
bi[np.nonzero(hydro_classes['cold_wds_ws_noperma'].values)] = 0.05
bi[np.nonzero(hydro_classes['cold_wds_cs_perma'].values)] = 0.3
bi[np.nonzero(hydro_classes['cold_wds_cs_noperma'].values)] = 0.5
bi[np.nonzero(hydro_classes['polar'].values)] = 0.1

params['infilt'] = xr.DataArray(bi,
                              dims=('nj', 'ni'),
                              coords={'xc': domain.xc, 'yc': domain.yc},
                              attrs={'description': "Fraction of maximum soil moisture where non-linear baseflow occurs", 
                                              'units': "fraction", 'long_name': "infilt"},
                                 encoding={"_FillValue": fillval_f,
                                           "Coordinates": "xc yc"})

soil depths (`depth`)

In [34]:
D1 = np.copy(masknan_vals)
D2 = np.copy(masknan_vals)
D3 = np.copy(masknan_vals)
D1[np.nonzero(domain.mask.values)] = 0.3
D3[np.nonzero(domain.mask.values)] = 0.5

D2[np.nonzero(hydro_classes['arid'].values)] = 2.0
D2[np.nonzero(hydro_classes['temperate_dry'].values)] = 2.0
D2[np.nonzero(hydro_classes['cold_dry_perma'].values)] = 0.5
D2[np.nonzero(hydro_classes['cold_dry_noperma'].values)] = 0.5
D2[np.nonzero(hydro_classes['cold_wds_ws_perma'].values)] = 2.0
D2[np.nonzero(hydro_classes['cold_wds_ws_noperma'].values)] = 0.5
D2[np.nonzero(hydro_classes['cold_wds_cs_perma'].values)] = 1.1
D2[np.nonzero(hydro_classes['cold_wds_cs_noperma'].values)] = 0.3
D2[np.nonzero(hydro_classes['polar'].values)] = 0.3

depths = np.rollaxis(np.dstack((D1, D2, D3)), axis=2)

params['depth'] = xr.DataArray(depths,
                                dims=('nlayer','nj', 'ni'),
                                coords={'xc': domain.xc, 'yc': domain.yc},
                                attrs={'description': "Thickness of each soil moisture layer",
                                   'units': "m", 'long_name': "depth"},
                                encoding={"_FillValue": fillval_f,
                                       "Coordinates": "xc yc"})

__aggregate ISRIC soil data to VIC soil depths__

first need array of soil depths 

In [35]:
soil_depths = params['depth'].sum(axis=0)
print("max soil depth is %.1f m" % soil_depths.max())

In [37]:
ksat_l1 = xr.apply_ufunc(calculate_first_layer_harmonic_mean,
                         ksat.isel(nlayer=0), 
                         ksat.isel(nlayer=1),
                         dask='allowed',
                         vectorize=True)

ksat_l2 = xr.apply_ufunc(calculate_second_layer_harmonic_mean,
                         ksat.isel(nlayer=2), 
                         ksat.isel(nlayer=3),
                         ksat.isel(nlayer=4),
                         ksat.isel(nlayer=5),
                         ksat.isel(nlayer=6),
                         soil_depths,
                         dask='allowed',
                         vectorize=True)

ksat_l3 = xr.apply_ufunc(calculate_second_layer_harmonic_mean,
                         ksat.isel(nlayer=2), 
                         ksat.isel(nlayer=3),
                         ksat.isel(nlayer=4),
                         ksat.isel(nlayer=5),
                         ksat.isel(nlayer=6),
                         soil_depths,
                         dask='allowed',
                         vectorize=True)

ksat_vals = np.rollaxis(np.dstack((ksat_l1, ksat_l2, ksat_l3)), 
                        axis=2)

params['Ksat'] = xr.DataArray(ksat_vals, 
                              dims=('nlayer','nj', 'ni'),
                              coords={'xc': domain.xc, 'yc': domain.yc},
                              attrs={'description': "Saturated hydraulic conductivity",
                                       'units': "mm/day", 'long_name': "Ksat"},
                              encoding={"_FillValue": fillval_f,
                                         "Coordinates": "xc yc"})

In [38]:
# bulk_density
bdm_l1 = xr.apply_ufunc(calculate_first_layer_arithmetic_mean,
                         bulk_density_min.isel(nlayer=0), 
                         bulk_density_min.isel(nlayer=1),
                         dask='allowed',
                         vectorize=True)

bdm_l2 = xr.apply_ufunc(calculate_second_layer_arithmetic_mean,
                         bulk_density_min.isel(nlayer=2), 
                         bulk_density_min.isel(nlayer=3),
                         bulk_density_min.isel(nlayer=4),
                         bulk_density_min.isel(nlayer=5),
                         bulk_density_min.isel(nlayer=6),
                         soil_depths,
                         dask='allowed',
                         vectorize=True)

bdm_l3 = xr.apply_ufunc(calculate_second_layer_arithmetic_mean,
                         bulk_density_min.isel(nlayer=2), 
                         bulk_density_min.isel(nlayer=3),
                         bulk_density_min.isel(nlayer=4),
                         bulk_density_min.isel(nlayer=5),
                         bulk_density_min.isel(nlayer=6),
                         soil_depths,
                         dask='allowed',
                         vectorize=True)

bdm_vals = np.rollaxis(np.dstack((bdm_l1, bdm_l2, bdm_l3)), 
                        axis=2)

params['bulk_density'] = xr.DataArray(bdm_vals,
                                      dims=('nlayer','nj', 'ni'),
                                      coords={'xc': domain.xc, 'yc': domain.yc},
                                      attrs={'description': "Mineral bulk density of soil layer",
                                             'units': "kg/m3", 'long_name': "bulk_density"},
                                      encoding={"_FillValue": fillval_f,
                                             "Coordinates": "xc yc"})

In [39]:
# expt
b_l1 = xr.apply_ufunc(calculate_first_layer_arithmetic_mean,
                         b.isel(nlayer=0), 
                         b.isel(nlayer=1),
                         dask='allowed',
                         vectorize=True)

b_l2 = xr.apply_ufunc(calculate_second_layer_arithmetic_mean,
                         b.isel(nlayer=2), 
                         b.isel(nlayer=3),
                         b.isel(nlayer=4),
                         b.isel(nlayer=5),
                         b.isel(nlayer=6),
                         soil_depths,
                         dask='allowed',
                         vectorize=True)

b_l3 = xr.apply_ufunc(calculate_second_layer_arithmetic_mean,
                         b.isel(nlayer=2), 
                         b.isel(nlayer=3),
                         b.isel(nlayer=4),
                         b.isel(nlayer=5),
                         b.isel(nlayer=6),
                         soil_depths,
                         dask='allowed',
                         vectorize=True)

expt_vals = np.rollaxis(np.dstack(((b_l1 * 2) + 3, (b_l2 * 2) + 3, (b_l3 * 2) + 3)), 
                        axis=2)

params['expt'] = xr.DataArray(expt_vals,
                                   dims=('nlayer','nj', 'ni'),
                                   coords={'xc': domain.xc, 'yc': domain.yc},
                                   attrs={'description': "Exponent n (=3+2/lambda) in Campbell's eqt for Ksat, HBH 5.6 \
                                           where lambda = soil pore size distribution parameter", 
                                           'units': "N/A", 'long_name': "expt"},
                                   encoding={"_FillValue": fillval_f,
                                             "Coordinates": "xc yc"})
params['bubble'] = xr.DataArray((np.copy(params['expt'].values) * 0.32) + 4.3,
                                         dims=('nlayer','nj', 'ni'),
                                         coords={'xc': domain.xc, 'yc': domain.yc},
                                         attrs={'description': "Bubbling pressure of soil. Values should be > 0",
                                           'units': "cm", 'long_name': "bubble"},
                                     encoding={"_FillValue": fillval_f,
                                               "Coordinates": "xc yc"})

In [40]:
# resid_moist
rm_l1 = xr.apply_ufunc(calculate_first_layer_arithmetic_mean,
                         resid_moist.isel(nlayer=0), 
                         resid_moist.isel(nlayer=1),
                         dask='allowed',
                         vectorize=True)

rm_l2 = xr.apply_ufunc(calculate_second_layer_arithmetic_mean,
                         resid_moist.isel(nlayer=2), 
                         resid_moist.isel(nlayer=3),
                         resid_moist.isel(nlayer=4),
                         resid_moist.isel(nlayer=5),
                         resid_moist.isel(nlayer=6),
                         soil_depths,
                         dask='allowed',
                         vectorize=True)

rm_l3 = xr.apply_ufunc(calculate_second_layer_arithmetic_mean,
                         resid_moist.isel(nlayer=2), 
                         resid_moist.isel(nlayer=3),
                         resid_moist.isel(nlayer=4),
                         resid_moist.isel(nlayer=5),
                         resid_moist.isel(nlayer=6),
                         soil_depths,
                         dask='allowed',
                         vectorize=True)
rm_vals = np.rollaxis(np.dstack((rm_l1, rm_l2, rm_l3)), 
                        axis=2)
params['resid_moist'] = xr.DataArray(rm_vals,
                                         dims=('nlayer','nj', 'ni'),
                                         coords={'xc': domain.xc, 'yc': domain.yc},
                                         attrs={'description': "Soil moisture layer residual moisture",
                                           'units': "fraction", 'long_name': "resid_moist"},
                                         encoding={"_FillValue": fillval_f,
                                               "Coordinates": "xc yc"})

In [41]:
# Wcr_FRACT
wcr_l1 = xr.apply_ufunc(calculate_first_layer_arithmetic_mean,
                         Wcr_FRACT.isel(nlayer=0), 
                         Wcr_FRACT.isel(nlayer=1),
                         dask='allowed',
                         vectorize=True)

wcr_l2 = xr.apply_ufunc(calculate_second_layer_arithmetic_mean,
                         Wcr_FRACT.isel(nlayer=2), 
                         Wcr_FRACT.isel(nlayer=3),
                         Wcr_FRACT.isel(nlayer=4),
                         Wcr_FRACT.isel(nlayer=5),
                         Wcr_FRACT.isel(nlayer=6),
                         soil_depths,
                         dask='allowed',
                         vectorize=True)

wcr_l3 = xr.apply_ufunc(calculate_second_layer_arithmetic_mean,
                         Wcr_FRACT.isel(nlayer=2), 
                         Wcr_FRACT.isel(nlayer=3),
                         Wcr_FRACT.isel(nlayer=4),
                         Wcr_FRACT.isel(nlayer=5),
                         Wcr_FRACT.isel(nlayer=6),
                         soil_depths,
                         dask='allowed',
                         vectorize=True)
wcr_vals = np.rollaxis(np.dstack((wcr_l1, wcr_l2, wcr_l3)), 
                        axis=2)

In [42]:
# Wpwp_FRACT
wpwp_l1 = xr.apply_ufunc(calculate_first_layer_arithmetic_mean,
                         Wpwp_FRACT.isel(nlayer=0), 
                         Wpwp_FRACT.isel(nlayer=1),
                         dask='allowed',
                         vectorize=True)

wpwp_l2 = xr.apply_ufunc(calculate_second_layer_arithmetic_mean,
                         Wpwp_FRACT.isel(nlayer=2), 
                         Wpwp_FRACT.isel(nlayer=3),
                         Wpwp_FRACT.isel(nlayer=4),
                         Wpwp_FRACT.isel(nlayer=5),
                         Wpwp_FRACT.isel(nlayer=6),
                         soil_depths,
                         dask='allowed',
                         vectorize=True)

wpwp_l3 = xr.apply_ufunc(calculate_second_layer_arithmetic_mean,
                         Wpwp_FRACT.isel(nlayer=2), 
                         Wpwp_FRACT.isel(nlayer=3),
                         Wpwp_FRACT.isel(nlayer=4),
                         Wpwp_FRACT.isel(nlayer=5),
                         Wpwp_FRACT.isel(nlayer=6),
                         soil_depths,
                         dask='allowed',
                         vectorize=True)

wpwp_vals = np.rollaxis(np.dstack((wpwp_l1, wpwp_l2, wpwp_l3)), 
                        axis=2)

In [43]:
# quartz
qz_l1 = xr.apply_ufunc(calculate_first_layer_arithmetic_mean,
                         quartz.isel(nlayer=0), 
                         quartz.isel(nlayer=1),
                         dask='allowed',
                         vectorize=True)

qz_l2 = xr.apply_ufunc(calculate_second_layer_arithmetic_mean,
                         quartz.isel(nlayer=2), 
                         quartz.isel(nlayer=3),
                         quartz.isel(nlayer=4),
                         quartz.isel(nlayer=5),
                         quartz.isel(nlayer=6),
                         soil_depths,
                         dask='allowed',
                         vectorize=True)

qz_l3 = xr.apply_ufunc(calculate_second_layer_arithmetic_mean,
                         quartz.isel(nlayer=2), 
                         quartz.isel(nlayer=3),
                         quartz.isel(nlayer=4),
                         quartz.isel(nlayer=5),
                         quartz.isel(nlayer=6),
                         soil_depths,
                         dask='allowed',
                         vectorize=True)
qz_vals = np.rollaxis(np.dstack((qz_l1, qz_l2, qz_l3)), 
                        axis=2)

params['quartz'] = xr.DataArray(qz_vals,
                                dims=('nlayer','nj', 'ni'),
                                coords={'xc': domain.xc, 'yc': domain.yc},
                                attrs={'description': "Quartz content of soil",
                                       'units': "cm", 'long_name': "quartz"},
                                encoding={"_FillValue": fillval_f,
                                       "Coordinates": "xc yc"})

In [44]:
# bulk_density
bd_l1 = xr.apply_ufunc(calculate_first_layer_arithmetic_mean,
                         soil_data['bulk_density']['bulk_density'].isel(nlayer=0), 
                         soil_data['bulk_density']['bulk_density'].isel(nlayer=1),
                         dask='allowed',
                         vectorize=True)

bd_l2 = xr.apply_ufunc(calculate_second_layer_arithmetic_mean,
                         soil_data['bulk_density']['bulk_density'].isel(nlayer=2), 
                         soil_data['bulk_density']['bulk_density'].isel(nlayer=3),
                         soil_data['bulk_density']['bulk_density'].isel(nlayer=4),
                         soil_data['bulk_density']['bulk_density'].isel(nlayer=5),
                         soil_data['bulk_density']['bulk_density'].isel(nlayer=6),
                         soil_depths,
                         dask='allowed',
                         vectorize=True)

bd_l3 = xr.apply_ufunc(calculate_second_layer_arithmetic_mean,
                         soil_data['bulk_density']['bulk_density'].isel(nlayer=2), 
                         soil_data['bulk_density']['bulk_density'].isel(nlayer=3),
                         soil_data['bulk_density']['bulk_density'].isel(nlayer=4),
                         soil_data['bulk_density']['bulk_density'].isel(nlayer=5),
                         soil_data['bulk_density']['bulk_density'].isel(nlayer=6),
                         soil_depths,
                         dask='allowed',
                         vectorize=True)
bd_vals = np.rollaxis(np.dstack((bd_l1, bd_l2, bd_l3)), 
                        axis=2)

params['bulk_density_comb'] = xr.DataArray(bd_vals,
                                      dims=('nlayer','nj', 'ni'),
                                      coords={'xc': domain.xc, 'yc': domain.yc},
                                      attrs={'description': "Soil bulk density of soil layer",
                                             'units': "kg/m3", 'long_name': "bulk_density"},
                                      encoding={"_FillValue": fillval_f,
                                             "Coordinates": "xc yc"})

In [45]:
#organic fract
of_l1 = xr.apply_ufunc(calculate_first_layer_arithmetic_mean,
                         soil_data['organic_fract']['organic_fract'].isel(nlayer=0), 
                         soil_data['organic_fract']['organic_fract'].isel(nlayer=1),
                         dask='allowed',
                         vectorize=True)

of_l2 = xr.apply_ufunc(calculate_second_layer_arithmetic_mean,
                         soil_data['organic_fract']['organic_fract'].isel(nlayer=2), 
                         soil_data['organic_fract']['organic_fract'].isel(nlayer=3),
                         soil_data['organic_fract']['organic_fract'].isel(nlayer=4),
                         soil_data['organic_fract']['organic_fract'].isel(nlayer=5),
                         soil_data['organic_fract']['organic_fract'].isel(nlayer=6),
                         soil_depths,
                         dask='allowed',
                         vectorize=True)

of_l3 = xr.apply_ufunc(calculate_second_layer_arithmetic_mean,
                         soil_data['organic_fract']['organic_fract'].isel(nlayer=2), 
                         soil_data['organic_fract']['organic_fract'].isel(nlayer=3),
                         soil_data['organic_fract']['organic_fract'].isel(nlayer=4),
                         soil_data['organic_fract']['organic_fract'].isel(nlayer=5),
                         soil_data['organic_fract']['organic_fract'].isel(nlayer=6),
                         soil_depths,
                         dask='allowed',
                         vectorize=True)
of_vals = np.rollaxis(np.dstack(((of_l1/1000), (of_l2/1000), (of_l3/1000))), 
                        axis=2)

params['organic'] = xr.DataArray(of_vals,
                                      dims=('nlayer','nj', 'ni'),
                                      coords={'xc': domain.xc, 'yc': domain.yc},
                                      attrs={'description': "soil organic carbon fraction",
                                             'units': "fraction", 'long_name': "organic_fract"},
                                      encoding={"_FillValue": fillval_f,
                                             "Coordinates": "xc yc"})

calculate porosity from bulk density and soil density 

In [46]:
sd_l1 = np.copy(masknan_vals)
sd_l2 = np.copy(masknan_vals)
sd_l3 = np.copy(masknan_vals)

sd_l1[np.nonzero(masknan_vals)] = 2685.0
sd_l2[np.nonzero(masknan_vals)] = 2685.0
sd_l3[np.nonzero(masknan_vals)] = 2685.0

sd_vals = np.rollaxis(np.dstack((sd_l1, sd_l2, sd_l3)), 
                        axis=2)
params['soil_density'] = xr.DataArray(sd_vals,
                                      dims=('nlayer','nj', 'ni'),
                                      coords={'xc': domain.xc, 'yc': domain.yc},
                                      attrs={'description': "Soil particle density, normally 2685 kg/m3",
                                       'units': "kg/m3", 'long_name': "soil_density"},
                                      encoding={"_FillValue": fillval_f,
                                           "Coordinates": "xc yc"})

In [47]:
sd_org_l1 = np.copy(masknan_vals)
sd_org_l2 = np.copy(masknan_vals)
sd_org_l3 = np.copy(masknan_vals)

sd_org_l1[np.nonzero(masknan_vals)] = 1300.0
sd_org_l2[np.nonzero(masknan_vals)] = 1300.0
sd_org_l3[np.nonzero(masknan_vals)] = 1300.0

sd_org_vals = np.rollaxis(np.dstack((sd_org_l1, sd_org_l2, sd_org_l3)), 
                        axis=2)
params['soil_density_org'] = xr.DataArray(sd_org_vals,
                                      dims=('nlayer','nj', 'ni'),
                                      coords={'xc': domain.xc, 'yc': domain.yc},
                                      attrs={'description': "Organic matter particle density, normally 1300 kg/m3",
                                       'units': "kg/m3", 'long_name': "soil_dens_org"},
                                      encoding={"_FillValue": fillval_f,
                                           "Coordinates": "xc yc"})

In [48]:
# calculate porosity
porosity = 1 - (params['bulk_density_comb'] / params['soil_density'])

In [49]:
params['Wpwp_FRACT'] = xr.DataArray(wpwp_vals / porosity.values,
                                         dims=('nlayer','nj', 'ni'),
                                         coords={'xc': domain.xc, 'yc': domain.yc},
                                         attrs={'description': "Fractional soil moisture content at the \
                                                wilting point (fraction of maximum moisture)",
                                           'units': "fraction", 'long_name': "Wpwp_FRACT"},
                                         encoding={"_FillValue": fillval_f,
                                               "Coordinates": "xc yc"})

params['Wcr_FRACT'] = xr.DataArray(wcr_vals / porosity.values,
                                         dims=('nlayer','nj', 'ni'),
                                         coords={'xc': domain.xc, 'yc': domain.yc},
                                         attrs={'description': "Fractional soil moisture content at the critical point \
                                                (~70%% of field capacity) (fraction of maximum moisture)",
                                           'units': "fraction", 'long_name': "Wcr_FRACT"},
                                         encoding={"_FillValue": fillval_f,
                                               "Coordinates": "xc yc"})

__make initial moisture fully saturated__

In [50]:
init_moist_l1 = xr.apply_ufunc(calculate_init_moist,
                               porosity.isel(nlayer=0), 
                               params.depth.isel(nlayer=0),
                               dask='allowed', 
                               vectorize=True)
init_moist_l2 = xr.apply_ufunc(calculate_init_moist,
                               porosity.isel(nlayer=1), 
                               params.depth.isel(nlayer=1),
                               dask='allowed', 
                               vectorize=True)
init_moist_l3 = xr.apply_ufunc(calculate_init_moist,
                               porosity.isel(nlayer=1), 
                               params.depth.isel(nlayer=2),
                               dask='allowed', 
                               vectorize=True)
init_moist_vals = np.rollaxis(np.dstack((init_moist_l1, init_moist_l2, init_moist_l3)), 
                        axis=2)
params['init_moist'] = xr.DataArray(init_moist_vals,
                                     dims=('nlayer','nj', 'ni'),
                                     coords={'xc': domain.xc, 'yc': domain.yc},
                                     attrs={'description': "Initial layer moisture content",
                                           'units': "mm", 'long_name': "init_moist"},
                                     encoding={"_FillValue": fillval_f,
                                             "Coordinates": "xc yc"})

__add: elev, c, phi_s, avg_T, dp, bubble, soil_density, off_gmt, rough, snow_rough,
annual_prec, fs_active__

In [51]:
params['off_gmt'] = xr.DataArray(old_params['off_gmt'].values,
                                 dims=('nj', 'ni'),
                                 coords={'xc': domain.xc, 'yc': domain.yc},
                                 encoding={"_FillValue": fillval_f,
                                               "Coordinates": "xc yc"})
params['phi_s'] = xr.DataArray(old_params['phi_s'].values,
                                    dims=('nlayer','nj', 'ni'),
                                    coords={'xc': domain.xc, 'yc': domain.yc},
                                    attrs={'description': "Soil moisture diffusion parameter",
                                           'units': "mm/mm", 'long_name': "phi_s"},
                                    encoding={"_FillValue": fillval_f,
                                              "Coordinates": "xc yc"})
params['fs_active'] = xr.DataArray(old_params['fs_active'].values,
                                dims=('nj', 'ni'),
                                coords={'xc': domain.xc, 'yc': domain.yc},
                                attrs={'description': "If set to 1, then frozen soil algorithm is activated for the \
                                        grid cell. A 0 indicates that frozen soils are not computed if soil \
                                        temperatures fall below 0C.", 
                                        'units': "binary", 'long_name': "fs_active"},
                                encoding={"_FillValue": fillval_i,
                                               "Coordinates": "xc yc", 'dtype': 'int32'})
params['dp'] = xr.DataArray(old_params['dp'].values,
                                dims=('nj', 'ni'),
                                coords={'xc': domain.xc, 'yc': domain.yc},
                                attrs={'description': "Soil thermal damping depth (depth at which soil temperature) \
                                        remains constant through the year, ~4 m", 
                                              'units': "m", 'long_name': "dp"},
                                encoding={"_FillValue": fillval_f,
                                               "Coordinates": "xc yc"})

In [52]:
snow_rough = np.copy(masknan_vals)
snow_rough[np.nonzero(masknan_vals)] = 0.0024
params['snow_rough'] = xr.DataArray(snow_rough,
                                    dims=('nj', 'ni'),
                                    coords={'xc': domain.xc, 'yc': domain.yc},
                                    attrs={'description': "Surface roughness of snowpack", 
                                              'units': "m", 'long_name': "snow_rough"},
                                    encoding={"_FillValue": fillval_f,
                                               "Coordinates": "xc yc"})

In [53]:
# add run_cell, mask, xv and yv, xc, yc, gridcell, lats, lons
params['run_cell'] = xr.DataArray(old_params['run_cell'].values, 
                                       dims=('nj', 'ni'),
                                       coords={'xc': domain.xc, 'yc': domain.yc},
                                       attrs={'units': "N/A", 'long_name': "run_cell"},
                                       encoding={"_FillValue": fillval_i,
                                                 "Coordinates": "xc yc",
                                                 "dtype": "int32"})
params['mask'] = xr.DataArray(domain['mask'].values,
                                   dims=('nj', 'ni'),
                                   coords={'xc': domain.xc, 'yc': domain.yc},
                                   attrs={'description': "0 value indicates cell is not active", 
                                          'units': "N/A", 'long_name': "mask", 'bounds': 'yv'},
                                   encoding={"_FillValue": fillval_i,
                                                 "Coordinates": "xc yc",
                                                 "dtype": "int32"})
params['gridcell'] = xr.DataArray(old_params['gridcell'].values, 
                                       dims=('nj', 'ni'),
                                       coords={'xc': domain.xc, 'yc': domain.yc},
                                       attrs={'description': "Grid cell number", 
                                              'units': "N/A", 'long_name': "gridcell"},
                                       encoding={"_FillValue": fillval_i,
                                                 "Coordinates": "xc yc", "dtype": "int32"})
params['lats'] = xr.DataArray(old_params.lats.values, 
                                   dims=('nj', 'ni'),
                                   coords={'xc': domain.xc, 'yc': domain.yc},
                                   attrs={'description': "Latitude of grid cell", 
                                              'units': "degrees", 'long_name': "lats"})
params['lons'] = xr.DataArray(old_params.lons.values,
                                   dims=('nj', 'ni'),
                                   coords={'xc': domain.xc, 'yc': domain.yc},
                                   attrs={'description': "Longitude of grid cell", 
                                          'units': "degrees", 'long_name': "lons"})
params['xc'] = xr.DataArray(domain['xc'].values,
                                 dims=('nj', 'ni'),
                                 attrs={'units': "degrees_east", 'long_name': "longitude of gridcell center",
                                        'bounds': 'xv'})
params['yc'] = xr.DataArray(domain['yc'].values,
                                 dims=('nj', 'ni'),
                                 attrs={'units': "degrees_north", 'long_name': "latitude of gridcell center",
                                        'bounds': 'yv'})

domain = domain.rename({'nv': 'nv4'})
params['xv'] = xr.DataArray(np.rollaxis(domain['xv'].values, axis=2),
                                 dims=('nv4', 'nj', 'ni'),
                                 attrs={'units': "degrees_east", 
                                        'long_name': "longitude of grid cell vertices"})

params['yv'] = xr.DataArray(np.rollaxis(domain['yv'].values, axis=2),
                                 dims=('nv4', 'nj', 'ni'),
                                 attrs={'units': "degrees_north", 
                                        'long_name': "latitude of grid cell vertices"})

swap 0th and 16th veg class to accommodate bare soil. 

veg class vars: Cv, Nveg, trunk_ratio, rarc, rmin, wind_h, RGL, rad_atten, wind_atten, albedo, LAI, overstory, displacement, veg_rough, root_depth, root_fract

additional organic fract parameters if that option is set to True: 

soil particle density of OM 

bulk density of OM

organic content of soil (fraction of total soil volume)

Note: Organic matter (%) = Total organic carbon (%) x 1.72, from http://www.soilquality.org.au/factsheets/organic-carbon

In [54]:
veg_class_vars = ['Cv', 'trunk_ratio', 'rarc', 'rmin', 'wind_h', 'RGL', 'rad_atten',
                  'wind_atten', 'albedo', 'LAI', 'overstory',
                  'root_depth', 'root_fract', 'displacement', 'veg_rough', 'max_snow_albedo']

for veg_class_var in veg_class_vars: 
    if ((veg_class_var == "LAI") or (veg_class_var == "albedo") or (veg_class_var == "root_depth") 
    or (veg_class_var == "root_fract") or (veg_class_var == "veg_rough") or 
        (veg_class_var == "displacement")):
        bare = np.copy(params[veg_class_var].isel(veg_class=0))
        last = np.copy(params[veg_class_var].isel(veg_class=16))
        params[veg_class_var].values[0, :, :, :] = last
        params[veg_class_var].values[16, :, :, :] = bare
    else:
        bare = np.copy(params[veg_class_var].isel(veg_class=0))
        last = np.copy(params[veg_class_var].isel(veg_class=16))
        params[veg_class_var].values[0, :, :] = last
        params[veg_class_var].values[16, :, :] = bare
params['root_fract'].values[16, :, :, :] = 0
params['root_depth'].values[16, :, :, :] = 0
params['displacement'].values[16, :, :, :] = 0
params['veg_rough'].values[16, :, :, :] = 0
params['overstory'].values[:, :, :] = 0

In [55]:
# adjust data vars that need adjusting 
params['Ksat'].values = params['Ksat'].where(domain.mask == 1)
params['expt'].values = params['expt'].where(domain.mask == 1)
params['bubble'].values = params['bubble'].where(domain.mask == 1)
params['Wpwp_FRACT'].values = params['Wpwp_FRACT'].where(domain.mask == 1)
params['Wcr_FRACT'].values = params['Wcr_FRACT'].where(domain.mask == 1)
params['resid_moist'].values = params['resid_moist'].where(domain.mask == 1)
params['quartz'].values = params['quartz'].where(domain.mask == 1)
params['bulk_density_comb'].values = params['bulk_density_comb'].where(domain.mask == 1)
params['bulk_density'].values = params['bulk_density'].where(domain.mask == 1)
params['soil_density'].values = params['soil_density'].where(domain.mask == 1)
params['c'].values = params['c'].where(domain.mask == 1)
params['snow_rough'].values = params['snow_rough'].where(domain.mask == 1)
params['Nveg'].values = params['Nveg'].where(domain.mask == 1)
params['trunk_ratio'] = params['trunk_ratio'].where(domain.mask == 1)
params['rarc'] = params['rarc'].where(domain.mask == 1)
params['rmin'] = params['rmin'].where(domain.mask == 1)
params['wind_h'] = params['wind_h'].where(domain.mask == 1)
params['RGL'] = params['RGL'].where(domain.mask == 1)
params['rad_atten'] = params['rad_atten'].where(domain.mask == 1)
params['wind_atten'] = params['wind_atten'].where(domain.mask == 1)
params['max_snow_albedo'] = params['max_snow_albedo'].where(domain.mask == 1)
params['root_depth'] = params['root_depth'].where(domain.mask == 1)
params['root_fract'] = params['root_fract'].where(domain.mask == 1)
params['albedo'] = params['albedo'].where(domain.mask == 1)
params['LAI'] = params['LAI'].where(domain.mask == 1)
params['overstory'] = params['overstory'].where(domain.mask == 1)
params['displacement'] = params['displacement'].where(domain.mask == 1)
params['veg_rough'] = params['veg_rough'].where(domain.mask == 1)
params['elev'] = params['elev'].where(domain.mask == 1)
params['avg_T'] = params['avg_T'].where(domain.mask == 1)
params['annual_prec'] = params['annual_prec'].where(domain.mask == 1)
params['rough'] = params['rough'].where(domain.mask == 1)
params['organic'] = params['organic'].where(domain.mask == 1)
params['soil_density_org'] = params['soil_density_org'].where(domain.mask == 1)

In [56]:
max_moist = params['depth'] * porosity * 1000
Wcr = params['Wcr_FRACT'] * max_moist
Wpwp = params['Wpwp_FRACT'] * max_moist
resid_moist_mm = params['resid_moist'] * params['depth'] * 1000

In [57]:
if (Wcr.where(Wpwp > Wcr).sum()) > 0:
    raise AssertionError("wilting point moisture is greater than critical point moisture")

In [58]:
if (params['resid_moist'].where(Wpwp < params['resid_moist']).sum()) > 0:
    raise AssertionError("wilting point moisture is less than residual moisture")

In [59]:
# Wpwp_FRACT MUST be >= resid_moist / (1.0 - bulk_density/soil_density).
if params['Wpwp_FRACT'].where(params['Wpwp_FRACT'] < params['resid_moist'] / porosity).sum() > 0:
    raise AssertionError("wilting point moisture is less than residual moisture")

In [60]:
encoding_params = {'run_cell': {'dtype': 'int32', "_FillValue": fillval_i}, 
                   'gridcell': {'dtype': 'int32', "_FillValue": fillval_i}, 
                   'fs_active': {'dtype': 'int32', "_FillValue": fillval_i}, 
                   'Nveg': {'dtype': 'int32', "_FillValue": fillval_i},
                   'overstory': {'dtype': 'int32', "_FillValue": fillval_i},
                   'veg_class': {'dtype': 'int32'}}
direc = '/u/home/gergel/data/parameters'

if organic_fract: 
    new_params_file = os.path.join(direc, 'new_vic5_params_organic_fract_20180930.nc')
else:
    new_params_file = os.path.join(direc, 'new_vic5_params_20181001.nc')
params.to_netcdf(new_params_file, format='NETCDF4_CLASSIC', encoding=encoding_params)

if organic_fract:
    print("Organic Fract is True, including soil_dens_org and organic_fract in parameters file")
    
print("saved new parameters to %s" %new_params_file)

saved new parameters to /u/home/gergel/data/parameters/new_vic5_params_20181001.nc
